Importing Model
=============

In [1]:
# 만들어진 모델을 불러오는데 필요한 코드
import pickle
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression as lgt
import re

# tokeinize - Twitter
from konlpy.tag import Twitter
twit = Twitter()

curDir = os.getcwd() # curDir = 현재 경로 / 폴더명1 / 폴더명 2 / 파일명.pkl

# model1
words1 = ['Adjective','Adverb','Conjunction','Exclamation','KoreanParticle','PreEomi','Eomi','Hashtag']
def tokenize1(doc):
    tagged = ['/'.join(t) for t in twit.pos(doc, norm=True, stem=True)]
    result = [word for word in tagged if word.split('/')[-1] in words1]
    return result
model1 = pickle.load(open(os.path.join(curDir,'model1.pkl'),'rb')) # call model
tfidf1 = pickle.load(open(os.path.join(curDir,'tfidf1.pkl'),'rb')) # call vectorizer

# model2
words2 = ['Noun','Josa','Adjective','Adverb','Exclamation','KoreanParticle','PreEomi','Eomi']
def tokenize2(doc):
    tagged = ['/'.join(t) for t in twit.pos(doc, norm=True, stem=True)]
    result = [word for word in tagged if word.split('/')[-1] in words2]
    return result
model2 = pickle.load(open(os.path.join(curDir,'model2.pkl'),'rb')) # call model
tfidf2 = pickle.load(open(os.path.join(curDir,'tfidf2.pkl'),'rb')) # call vectorizer

# model3
def tokenize3(doc):
    tagged = ['/'.join(t) for t in twit.pos(doc, norm=True, stem=True)]
    result = [word for word in tagged if word.split('/')[-1]]
    return result

model3 = pickle.load(open(os.path.join(curDir,'model3.pkl'),'rb')) # call model
tfidf3 = pickle.load(open(os.path.join(curDir,'tfidf3.pkl'),'rb')) # call vectorizer

Read Data_Set
=============

In [7]:
import pandas as pd
# path에 파일명 입력
path = r'.\raw_data\101_twit.xlsx'
original_file = path[path.rfind('\\'):]
df_test=pd.read_table(path)
h_x = df_test['contents'].as_matrix()

KeyError: 'contents'

In [3]:
import pandas as pd
path = r'.\raw_data\instagram_notadv.txt'
df_test=pd.read_table(path,delimiter='\t')
original_file = path[path.rfind('\\'):path.rfind('.')]
h_x = df_test['contents'].as_matrix()

### Processing...

In [5]:
def score(model,tfidf,input_data):
    target = tfidf.transform(h_x)
    result = model.predict(target)
    result_p = model.predict_proba(target)
    df_prob = pd.DataFrame(result_p)
    result_p0 = df_prob[0]
    result_p1 = df_prob[1]
    point = round((result_p1 - result_p0),2)*100
    return point

In [ ]:
score1 = score(model1,tfidf1,h_x) # n_gram1
score2 = score(model2,tfidf2,h_x) # n_gram2
score3 = score(model3,tfidf3,h_x) # n_gram3

In [ ]:
result = score1/2 + score2/4 + score3/4
label = []
for i in  range(len(result)):
    if result[i] >= 50:
        label.append('POS')
    elif result[i] > -50:
        label.append('NEU')
    else:
        label.append('NEG')

Output Data
=============

In [16]:
data = {'point':result,'predict_label':label}
labeled = pd.DataFrame(data)
output = df_test.join(labeled,lsuffix='_df_test', rsuffix='labeled')

In [18]:
output.to_excel('predicted_'+original_file,encoding='utf-8')

C:\ProgramData\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:830: UserWarning: Ignoring URL 'https://form.office.naver.com/form/responseViewMobile.cmd?formkey=ZWQ1YzM1ZTAtODA1Mi00ZDM5LWI2YTctNjU5MDU4MDMwOTI2&sourceId=urlshare%20…%20%3c워너원%20보틀%20&%20떡메%20주문%20받습니다%3e%20폼에서%20설명%20찬찬히%20잘%20읽어보시고%20신중한%20주문부탁드립니다!%20-리트윗%20하신%20분들%20중%20추첨을%20통해%203분께%20무료나눔%20하고자%20합니다!!!%20' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
